# Deployment Preparation & Verification

This notebook simulates the deployment environment and verifies the application logic.

## Objectives:
- Verify `app/streamlit_app.py` dependencies
- Test model loading in a simulated production environment
- Generate sample predictions
- Check `submission.json` validity

In [1]:
import os
import sys
import joblib
import json
import pandas as pd
import numpy as np

print("✓ Environment check passed")

✓ Environment check passed


## 1. Verify Model Existence

In [2]:
required_files = [
    '../models/best_model.pkl',
    '../data/processed/feature_names.json',
    '../models/scaler.pkl',
    '../data/processed/feature_info.json',
    '../submission.json'
]

missing = []
for f in required_files:
    if not os.path.exists(f):
        print(f"❌ Missing: {f}")
        missing.append(f)
    else:
        print(f"✓ Found: {f}")

if missing:
    print("\nCRITICAL: Missing deployment files!")
else:
    print("\n✓ All deployment files present")

✓ Found: ../models/best_model.pkl
✓ Found: ../data/processed/feature_names.json
✓ Found: ../models/scaler.pkl
✓ Found: ../data/processed/feature_info.json
✓ Found: ../submission.json

✓ All deployment files present


## 2. Simulate Prediction Request

In [3]:
# Load model artifacts
model = joblib.load('../models/best_model.pkl')
scaler = joblib.load('../models/scaler.pkl')
with open('../data/processed/feature_names.json', 'r') as f:
    features = json.load(f)['feature_names']

print(f"Model: {type(model).__name__}")
print(f"Expected Features: {len(features)}")

# Create dummy input
dummy_input = pd.DataFrame(np.random.rand(1, len(features)), columns=features)
dummy_scaled = scaler.transform(dummy_input)

# Predict
try:
    pred = model.predict(dummy_scaled)
    prob = model.predict_proba(dummy_scaled)
    
    print(f"\nPrediction: {pred[0]}")
    print(f"Probability: {prob[0]}")
    print("✓ Inference test passed")
except Exception as e:
    print(f"❌ Inference failed: {e}")

Model: XGBClassifier
Expected Features: 39

Prediction: 1
Probability: [0.26207888 0.7379211 ]
✓ Inference test passed


## 3. Validate Submission JSON

In [4]:
try:
    with open('../submission.json', 'r') as f:
        submission = json.load(f)

    print(json.dumps(submission, indent=4))

    # Validation checks
    if submission.get('roc_auc', 0) < 0.75:
        print("\n⚠️  ROC-AUC matches requirement?")
    else:
        print("\n✓ ROC-AUC meets requirement")
except FileNotFoundError:
    print("⚠️  submission.json not found yet")

{
    "student_info": {
        "name": "Rushikesh",
        "email": "rushikeshkunisetty@gmail.com",
        "student_id": "23MH1A4930",
        "submission_date": "2026-02-10"
    },
    "repository": {
        "github_url": "https://github.com/Rushikesh-5706/Customer-Churn-Prediction-System",
        "commit_hash": "latest-commit-hash"
    },
    "deployment": {
        "live_url": "https://customer-churn-prediction.streamlit.app",
        "platform": "Streamlit Cloud",
        "deployment_date": "2026-02-10",
        "status": "Ready for Deployment"
    },
    "dataset": {
        "source": "UCI Machine Learning Repository",
        "url": "http://archive.ics.uci.edu/ml/datasets/Online+Retail+II",
        "original_rows": 525461,
        "cleaned_rows": 342273,
        "final_customers": 3213
    },
    "models": {
        "models_trained": [
            "Logistic Regression",
            "Decision Tree",
            "Random Forest",
            "Gradient Boosting",
            "Ne